In [29]:
# ------------------------------------------------------------------------------ #
# @Author:        F. Paul Spitzner
# @Email:         paul.spitzner@ds.mpg.de
# @Created:       2024-03-23 16:05:20
# @Last Modified: 2024-03-23 16:05:38
# ------------------------------------------------------------------------------ #
# Here we run the bayesian analysis.
# Final plotting is done in a different notebook, as the analysis takes quite
# some time.
# ------------------------------------------------------------------------------ #

# Files we need to migrate:
# - experiment/legacy/plotting/allen_hierarchical_bayes[...].py

import os
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import patsy as pt
import scipy.stats as st
import arviz as az

import logging

logging.basicConfig(
    format="%(asctime)s | %(levelname)-8s | %(name)-s | %(funcName)-s | %(message)s",
    level=logging.WARNING,
)
log = logging.getLogger("notebook")
log.setLevel("DEBUG")

extra_path = os.path.abspath("../")
sys.path.append(extra_path)
log.info(f"project directory: {extra_path}")

from ana import utility as utl
from ana import plot_helper as ph

ph.log.setLevel("DEBUG")
utl.log.setLevel("DEBUG")

data_dir = "/Users/paul/para/5_Archive/information_timescales/repo/gnode.nosync/experiment_analysis/dat"

# load our results from timescale fitting
df = pd.read_hdf(f"{data_dir}/all_units_merged_blocks.h5", key="meta_df")

# add metrics from image selectivity etc.
df = utl.load_metrics(
    df.reset_index(),
    data_dir,
    csvs=[
        "brain_observatory_1.1_analysis_metrics.csv",
        "functional_connectivity_analysis_metrics.csv",
    ],
    cols=["on_screen_rf"],
)
df = utl.add_structure_and_hierarchy_scores(df)
df

2024-05-10 16:09:45,692 | INFO     | notebook | <module> | project directory: /Users/paul/para/5_Archive/information_timescales/repo/_latest/experiment_analysis
2024-05-10 16:09:48,145 | DEBUG    | its_utility | load_metrics | Loaded columns ['unit_id', 'on_screen_rf'] from /Users/paul/para/5_Archive/information_timescales/repo/gnode.nosync/experiment_analysis/dat/brain_observatory_1.1_analysis_metrics.csv
2024-05-10 16:09:48,373 | DEBUG    | its_utility | load_metrics | Loaded columns ['unit_id', 'on_screen_rf'] from /Users/paul/para/5_Archive/information_timescales/repo/gnode.nosync/experiment_analysis/dat/functional_connectivity_analysis_metrics.csv
2024-05-10 16:09:48,373 | INFO     | its_utility | load_metrics | Column on_screen_rf found in multiple dataframes.
2024-05-10 16:09:48,375 | DEBUG    | its_utility | load_metrics | Matched 6368 rows from meta_df in /Users/paul/para/5_Archive/information_timescales/repo/gnode.nosync/experiment_analysis/dat/brain_observatory_1.1_analysis_

,index,session,stimulus,block,unit_id,ecephys_structure_acronym,invalid_spiketimes_check,recording_length,firing_rate,filepath,...,R_tot,tau_R,tau_single,tau_double,tau_R_details,tau_single_details,tau_double_details,on_screen_rf,structure_name,hierarchy_score
0,0,715093703,natural_movie_three,merged_3.0_and_6.0,950911932,VISam,SUCCESS,1078.491211,8.025100,/project.nst/neuroscience-raw/Allen/visual_cod...,...,0.074624,0.058510,0.000054,0.748946,"{'firing_rate': 8.012090922999178, 'firing_rat...","{'tau': 5.404084198337884e-05, 'mre': 6.575654...","{'tau': 0.7489462162647652, 'mre': 0.993346188...",False,AM,0.441
1,1,715093703,natural_movie_three,merged_3.0_and_6.0,950911986,VISam,SUCCESS,1079.203369,3.647135,/project.nst/neuroscience-raw/Allen/visual_cod...,...,0.112344,0.088701,2.302634,1.874703,"{'firing_rate': 3.6471291367256455, 'firing_ra...","{'tau': 2.3026338728859774, 'mre': 0.997830929...","{'tau': 1.874703126979266, 'mre': 0.9973364645...",False,AM,0.441
2,2,715093703,natural_movie_three,merged_3.0_and_6.0,950912164,VISam,SUCCESS,1077.872803,8.500075,/project.nst/neuroscience-raw/Allen/visual_cod...,...,0.065517,0.059416,1.026283,1.026284,"{'firing_rate': 8.475008697669026, 'firing_rat...","{'tau': 1.0262833654998276, 'mre': 0.995139899...","{'tau': 1.0262836602024297, 'mre': 0.995139901...",True,AM,0.441
3,3,715093703,natural_movie_three,merged_3.0_and_6.0,950912190,VISam,SUCCESS,1075.171875,4.399297,/project.nst/neuroscience-raw/Allen/visual_cod...,...,0.093197,0.116100,1.564297,1.363502,"{'firing_rate': 4.395563512916502, 'firing_rat...","{'tau': 1.5642971952849982, 'mre': 0.996808779...","{'tau': 1.363502294341311, 'mre': 0.9963396881...",True,AM,0.441
4,4,715093703,natural_movie_three,merged_3.0_and_6.0,950912214,VISam,SUCCESS,1073.492432,5.326540,/project.nst/neuroscience-raw/Allen/visual_cod...,...,0.118505,0.064748,0.605686,0.377801,"{'firing_rate': 5.317211538013684, 'firing_rat...","{'tau': 0.6056864214354767, 'mre': 0.991778883...","{'tau': 0.37780093267583975, 'mre': 0.98685270...",True,AM,0.441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11922,11922,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190716,LP,SUCCESS,1669.521729,3.438110,/project.nst/neuroscience-raw/Allen/visual_cod...,...,0.082713,0.057480,0.154983,0.147879,"{'firing_rate': 3.3512526017879334, 'firing_ra...","{'tau': 0.15498293185242107, 'mre': 0.96825323...","{'tau': 0.1478788520828995, 'mre': 0.966753758...",False,LP,0.105
11923,11923,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190722,LP,SUCCESS,1677.163818,2.705758,/project.nst/neuroscience-raw/Allen/visual_cod...,...,0.061618,0.179228,0.287762,0.248368,"{'firing_rate': 2.6944277992922583, 'firing_ra...","{'tau': 0.2877620706377681, 'mre': 0.982774617...","{'tau': 0.24836808551088616, 'mre': 0.98006987...",False,LP,0.105
11924,11924,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190724,LP,SUCCESS,1676.766846,2.142814,/project.nst/neuroscience-raw/Allen/visual_cod...,...,0.075886,0.067158,0.345960,0.312267,"{'firing_rate': 2.132075359172695, 'firing_rat...","{'tau': 0.34596009410375744, 'mre': 0.98565140...","{'tau': 0.3122666989915499, 'mre': 0.984115555...",False,LP,0.105
11925,11925,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190819,LP,SUCCESS,1675.092041,1.989741,/project.nst/neuroscience-raw/Allen/visual_cod...,...,0.022345,0.432975,0.326839,0.326856,"{'firing_rate': 1.9873499711956635, 'firing_ra...","{'tau': 0.3268387313372349, 'mre': 0.984818356...","{'tau': 0.3268561287459293, 'mre': 0.984819158...",False,LP,0.105


In [30]:
df["on_screen_rf"].isna().sum()

0

In [31]:
# pymc does not like bools:
df["on_screen_rf"] = df["on_screen_rf"].astype("int")

In [32]:
# Log transforms
cols_to_trafo = ["firing_rate", "R_tot", "tau_R", "tau_double"]
num_rows_before = len(df)
for c in cols_to_trafo:
    df[f"log_{c}"] = np.log(df[c])

df = df.dropna(subset=[f"log_{c}" for c in cols_to_trafo])
num_rows_after = len(df)
log.info(f"dropped {num_rows_before - num_rows_after} rows due to inf/nans")

# helper functions to go back and forth, we will use them in pymc to add deterministic variables
_log_sd = dict()
_log_mean = dict()
for c in cols_to_trafo:
    assert df[f"log_{c}"].isna().sum() == 0
    _log_sd[c] = df[f"log_{c}"].std()
    _log_mean[c] = df[f"log_{c}"].mean()

def f_transf_int(c, x):
    res = np.exp(x * _log_sd[c] + _log_mean[c])
    if c in ["tau_R", "tau_double"]:
        res *= 1000
    return res

def f_transf_int_inverse(c, x):
    if c in ["tau_R", "tau_double"]:
        x = np.divide(x, 1000)
    return (np.log(x) - _log_mean[c]) / _log_sd[c]


2024-05-10 16:10:08,827 | INFO     | notebook | <module> | dropped 14 rows due to inf/nans


In [33]:
# z-transfrom
# -----------
# and mean-centered & standardized according to Gelman: 1 / (2 * sd)
# Divide by 2 standard deviations in order to put the variance of a
# normally distributed variable nearer to the variance range of a
# binary variable. See
# http://www.stat.columbia.edu/~gelman/research/published/standardizing7.pdf
# for more info.

def z_trafo(col):
    return (col - col.mean()) / col.std()

def gelman_mean(col):
    return (col - col.mean()) / 2.0 / col.std()

def gelman_min(col):
    return (col - col.min()) / 2.0 / col.std()

for c in ["R_tot", "tau_R", "tau_double"]:
    df[f"z_log_{c}"] = z_trafo(df[f"log_{c}"])

for c in ["firing_rate"]:
    df[f"z_log_{c}"] = gelman_mean(df[f"log_{c}"])

for c in ["hierarchy_score"]:
    df[f"z_{c}"] = gelman_min(df[f"{c}"])


In [34]:
# to model session-level details, we need a lookup
# row -> session_idx
session_idx, sessions = pd.factorize(df["session"])
df["session_idx"] = session_idx
num_sessions = len(sessions)

In [24]:
class LinearModel(pm.Model):
    def __init__(self, measure="R_tot", name=""):
        super().__init__(name=name, model=None)

        # hyperpriors
        mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=1.0)
        sigma_intercept = pm.HalfCauchy("sigma_intercept", beta=1.0)

        mu_slope = pm.Normal("mu_slope", mu=0.0, sigma=1.0)
        sigma_slope = pm.HalfCauchy("sigma_slope", beta=1.0)

        session_intercept = pm.Normal(
            "session_intercept",
            mu=0.0,
            sigma=1.0,
            shape=num_sessions,
        )

        session_slope = pm.Normal(
            "session_slope",
            mu=0.0,
            sigma=1.0,
            shape=num_sessions,
        )

        b_os_rf = pm.Normal("b_os_rf", mu=0.0, sigma=1.0)
        b_log_fr = pm.Normal("b_log_fr", mu=0.0, sigma=1.0)

        # deterministic variables. we do not use them for furhter modeling, but they help visualizing.
        pm.Deterministic(
            "eff_session_hierarchy_slope", mu_slope + session_slope * sigma_slope
        )
        pm.Deterministic(
            "eff_session_intercept", mu_intercept + session_intercept * sigma_intercept
        )
        pm.Deterministic(
            "linear_fit",
            f_transf_int(
                measure,
                mu_intercept + mu_slope * df["z_hierarchy_score"].values,
            ),
        )

        # linear model
        yest = (
            # global intercept
            mu_intercept
            # session-level intercept. one for each sessions
            + sigma_intercept * session_intercept[session_idx]
            # session-level slope x neuron hierarchy score
            + (mu_slope + sigma_slope * session_slope[session_idx])
            * df["z_hierarchy_score"].values
            # per-unit terms
            + b_os_rf * df["on_screen_rf"].values
            + b_log_fr * df["z_log_firing_rate"].values
        )

        # define normal likelihood with halfnormal error
        epsilon = pm.HalfCauchy("epsilon", 10.0)

        if measure == "R_tot":
            likelihood = pm.Normal(
                "likelihood",
                mu=yest,
                sigma=epsilon,
                observed=df[f"z_log_{measure}"],
            )
        else:
            alpha = pm.Normal("alpha", mu=0.0, sigma=1.0)
            likelihood = pm.SkewNormal(
                "likelihood",
                mu=yest,
                sigma=epsilon,
                alpha=alpha,
                observed=df[f"z_log_{measure}"],
            )

lm_R_tot = LinearModel("R_tot")
print(lm_R_tot.str_repr())

               mu_intercept ~ Normal(0, 1)
            sigma_intercept ~ HalfCauchy(0, 1)
                   mu_slope ~ Normal(0, 1)
                sigma_slope ~ HalfCauchy(0, 1)
          session_intercept ~ Normal(0, 1)
              session_slope ~ Normal(0, 1)
                    b_os_rf ~ Normal(0, 1)
                   b_log_fr ~ Normal(0, 1)
                    epsilon ~ HalfCauchy(0, 10)
eff_session_hierarchy_slope ~ Deterministic(f(session_slope, mu_slope, sigma_slope))
      eff_session_intercept ~ Deterministic(f(session_intercept, mu_intercept, sigma_intercept))
                 linear_fit ~ Deterministic(f(mu_intercept, mu_slope))
                 likelihood ~ Normal(f(b_log_fr, b_os_rf, mu_intercept, mu_slope, session_intercept, sigma_intercept, session_slope, sigma_slope), epsilon)


In [ ]:
# TODO: compare `eff_session_hierarchy_slope` between spontaneous and natural movie three

In [14]:
with lm_R_tot:
    idata = pm.sample_prior_predictive(samples=500, random_seed=42)

Sampling: [b_log_fr, b_os_rf, epsilon, likelihood, mu_intercept, mu_slope, session_intercept, session_slope, sigma_intercept, sigma_slope]
2024-05-02 19:44:28,765 | INFO     | pymc.sampling.forward | sample_prior_predictive | Sampling: [b_log_fr, b_os_rf, epsilon, likelihood, mu_intercept, mu_slope, session_intercept, session_slope, sigma_intercept, sigma_slope]


In [36]:
# for the structure group model, we need to select data from higher cortical and thalamus, respectively.
# add a column each that is (1) if the neuron is in this group else (0)

df["is_higher_cortical"] = (
    df["structure_name"].isin(["LM", "RL", "AL", "PM", "AM"]).astype("int")
)
df["is_thalamus"] = df["structure_name"].isin(["LGN", "LP"]).astype("int")

print(df["is_higher_cortical"].value_counts())
print(df["is_thalamus"].value_counts())

is_higher_cortical
1    7424
0    4489
Name: count, dtype: int64
is_thalamus
0    9732
1    2181
Name: count, dtype: int64


In [18]:
class StructureGroupModel(pm.Model):
    def __init__(self, measure="R_tot", name=""):
        super().__init__(name=name, model=None)

        # hyperpriors
        mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=1.0)
        # PS 24-05-02: prior inconsistent with lm?
        sigma_intercept = pm.HalfCauchy("sigma_intercept", beta=0.1)

        # higher cortical
        mu_hc_offset = pm.Normal("mu_hc_offset", mu=0.0, sigma=1.0)
        sigma_hc_offset = pm.HalfCauchy("sigma_hc_offset", beta=1.0)

        # thalamus
        mu_th_offset = pm.Normal("mu_th_offset", mu=0.0, sigma=1.0)
        sigma_th_offset = pm.HalfCauchy("sigma_th_offset", beta=1.0)

        session_intercept = pm.Normal(
            "session_intercept",
            mu=0.0,
            sigma=1.0,
            shape=num_sessions,
        )
        session_hc_offset = pm.Normal(
            "session_hc_offset", mu=0.0, sigma=1.0, shape=num_sessions
        )
        session_th_offset = pm.Normal(
            "session_th_offset", mu=0.0, sigma=1.0, shape=num_sessions
        )

        b_os_rf = pm.Normal("b_os_rf", mu=0.0, sigma=1.0)
        b_log_fr = pm.Normal("b_log_fr", mu=0.0, sigma=1.0)

        # deterministic variables. we do not use them for furhter modeling, but they help visualizing.
        pm.Deterministic(
            "eff_session_hc_offset", mu_hc_offset + session_hc_offset * sigma_hc_offset
        )
        pm.Deterministic(
            "eff_session_th_offset", mu_th_offset + session_th_offset * sigma_th_offset
        )
        pm.Deterministic(
            "eff_session_intercept", mu_intercept + session_intercept * sigma_intercept
        )

        # structure group model
        yest = (
            # global intercept
            mu_intercept
            # session-level intercept. one for each sessions
            + sigma_intercept * session_intercept[session_idx]
            #
            + (mu_hc_offset + session_hc_offset[session_idx] * sigma_hc_offset)
            * df["is_higher_cortical"].values
            #
            + (mu_th_offset + session_th_offset[session_idx] * sigma_th_offset)
            * df["is_thalamus"].values
            # per-unit terms
            + b_os_rf * df["on_screen_rf"].values
            + b_log_fr * df["z_log_firing_rate"].values
        )

        # define normal likelihood with halfnormal error
        epsilon = pm.HalfCauchy("epsilon", 10.0)

        if measure == "R_tot":
            likelihood = pm.Normal(
                "likelihood",
                mu=yest,
                sigma=epsilon,
                observed=df[f"z_log_{measure}"],
            )
        else:
            alpha = pm.Normal("alpha", mu=0.0, sigma=1.0)
            likelihood = pm.SkewNormal(
                "likelihood",
                mu=yest,
                sigma=epsilon,
                alpha=alpha,
                observed=df[f"z_log_{measure}"],
            )


sgm_R_tot = StructureGroupModel("R_tot")
print(sgm_R_tot.str_repr())

         mu_intercept ~ Normal(0, 1)
      sigma_intercept ~ HalfCauchy(0, 0.1)
         mu_hc_offset ~ Normal(0, 1)
      sigma_hc_offset ~ HalfCauchy(0, 1)
         mu_th_offset ~ Normal(0, 1)
      sigma_th_offset ~ HalfCauchy(0, 1)
    session_intercept ~ Normal(0, 1)
    session_hc_offset ~ Normal(0, 1)
    session_th_offset ~ Normal(0, 1)
              b_os_rf ~ Normal(0, 1)
             b_log_fr ~ Normal(0, 1)
              epsilon ~ HalfCauchy(0, 10)
eff_session_hc_offset ~ Deterministic(f(session_hc_offset, mu_hc_offset, sigma_hc_offset))
eff_session_th_offset ~ Deterministic(f(session_th_offset, mu_th_offset, sigma_th_offset))
eff_session_intercept ~ Deterministic(f(session_intercept, mu_intercept, sigma_intercept))
           likelihood ~ Normal(f(b_log_fr, b_os_rf, mu_th_offset, mu_intercept, sigma_th_offset, session_th_offset, mu_hc_offset, session_intercept, sigma_intercept, sigma_hc_offset, session_hc_offset), epsilon)


In [ ]:
%load_ext watermark
%watermark -v --iversions --packages mrestimator,hdestimator,scipy

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.22.2

mrestimator: 0.1.9b2
hdestimator: 0.10b2
scipy      : 1.13.0

patsy         : 0.5.6
scipy         : 1.13.0
logging       : 0.5.1.2
prompt_toolkit: 3.0.42
sys           : 3.11.9 | packaged by conda-forge | (main, Apr 19 2024, 18:34:54) [Clang 16.0.6 ]
pymc          : 5.14.0
matplotlib    : 3.8.4
pandas        : 2.0.3
numpy         : 1.24.4
sqlite3       : 2.6.0
IPython       : 8.22.2
arviz         : 0.18.0

